In [94]:
import numpy as np
import pandas as pd
import implicit 
import scipy

df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)

data = df.drop(['Unnamed: 0'], axis=1)

users = list(np.sort(data.user_id.unique()))
artists = list(data.artist_name.unique())
plays = list(data.plays)

rows = data.user_id.astype('category', categories=users).cat.codes
cols = data.artist_mbid.astype('category', categories=artists).cat.codes
plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))

plays_sparse

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  from ipykernel import kernelapp as app


ValueError: negative column index found

In [95]:
plays_sparse[0]

<1x47102 sparse matrix of type '<class 'numpy.int64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [96]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

In [97]:
# train the model on a sparse matrix of item/user/confidence weights
model.fit(plays_sparse)

100%|██████████| 15.0/15 [00:05<00:00,  3.11it/s]


In [98]:
# recommend items for a user
user_items = plays_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

NameError: name 'userid' is not defined

In [116]:
# find related items
dummy=27
related = model.similar_items(dummy)

In [117]:
print("These are the artists similar to ",artists[dummy])
for (x,y) in related:
    print(artists[x])

These are the artists similar to  beirut
beirut
say hi to your mom
october falls
sister machine gun
skullview
margot & the nuclear so and so's
b-war & lukestar
path of resistance
blaze
mdc
